<a href="https://colab.research.google.com/github/kinslersi/colab/blob/main/%E7%88%AC%E8%9F%B2%E7%AC%AC%E4%B8%80%E9%A1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install selenium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buste...

Executing: /tmp/apt-key-gpghome.svYhtM5sa8/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.R5KiC5ygBO/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.x91yC4OEGu/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
gpg: cannot open '/dev/tty': No such device or address
gpg: [stdout]: write error: Broken pipe
gpg: filter_flush failed on c

CalledProcessError: ignored

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
import time
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from selenium.webdriver.chrome.options import Options

user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.0.3 Safari/605.1.15"
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--user-agent=%s' % user_agent)
driver = webdriver.Chrome('chromedriver',options=chrome_options)

In [ ]:
# 生成各校資料裡每一row ID
def produce_line_id(first_num,second_num):
    if second_num<10:
        update_line=str(0)+str(0)+str(second_num)
    elif second_num<100:
        update_line=str(0)+str(second_num)
    else:
        update_line=str(second_num)
    line_id="allDeptTr_"+str(first_num)+str(update_line)
    return line_id

In [ ]:
# 分割處理part
def processing(school_name,line):
    print('getting data from: ',school_name)
    while True:
        line+=1
        try:    
            row=driver.find_element(By.ID,produce_line_id(school_code,line)).text
        # 到底了沒資料
        except NoSuchElementException:   
            break
        else:
            clean_row=row.split('-')[2].split(" ")
            # num0: 科系
            num0=clean_row[1]
            # num1: 類別
            num1=clean_row[-5]
            # num2: 一般考生 
            num2=int(clean_row[-4])
            # num3: 低收或中低考生
            num3=int(clean_row[-3])
            # 原住民考生
            num4=int(clean_row[-2])
            # num5: 離島考生
            num5=int(clean_row[-1])

            # 各校各類考生錄取人數
            school_sepdata.append([num0+f'({num1})',num2,num3,num4,num5])
            # 各校系的招生類別
            school_mergedata.append([num1,num2+num3+num4+num5])        
    return school_sepdata,school_mergedata

In [ ]:
# main.py
website_path=input("網址： ")
data=driver.get(website_path)
time.sleep(3)
# 找尋左上角區位block
driver.find_element(By.XPATH,'//*[@id="areaSelect"]')    
block=Select(driver.find_element(By.NAME,'areaSelect'))
# block選不限制
block.select_by_index(1)                                 
time.sleep(1)  
# 網頁loading
WebDriverWait(driver,10).until(                            
        EC.presence_of_element_located((By.ID,"resultListDiv"))).text.split("-")[1]

# 每間學校selenium
for i in range(1,127):
    # 各校系的招生類別
    school_mergedata=[]            
    # 各校各類考生錄取人數 
    school_sepdata=[]              
    # 學校右方細節loading
    school_code,school_name=WebDriverWait(driver,10).until(        
        EC.presence_of_element_located((By.XPATH,f"/html/body/table/tbody/tr/td[1]/div/div[3]/div/div[2]/div[{i}]"))).text.split("-")
    school_code=school_code.strip()
    driver.find_element(By.XPATH,f"/html/body/table/tbody/tr/td[1]/div/div[3]/div/div[2]/div[{i}]").click()

    line1=0
    line2=400
    school_sepdata,school_mergedata=processing(school_name,line1)
    # 檢查是否存在東西不然需要換另一個參數帶入processing
    if school_sepdata==[] and school_mergedata==[]:
        school_sepdata,school_mergedata=processing(school_name,line2)
    # 該校各類別所有學生(一般,低收或中低收,原住民,離島)
    df1=pd.DataFrame(school_sepdata,columns=["校系",'一般考生','低收或中低考生','原住民考生','離島考生'])
    # df2_sum各類考生錄取人數加總
    df2=pd.DataFrame(school_mergedata,columns=["招生類別","錄取人數"])
    df2_sum=df2.groupby('招生類別',as_index=False).sum()    

    # 輸出csv檔
    df1.to_csv(f'{school_name}_招生類別.csv',sep=",",index=True,header=True)
    df2_sum.to_csv(f'{school_name}_錄取人數.csv',sep=",",index=False,header=True)
driver.close()
print("done")

網址： https://www.jctv.ntut.edu.tw/downloads/112/apply/ugcdrom/index.html
getting data from:   國立臺灣科技大學
getting data from:   國立雲林科技大學
getting data from:   國立屏東科技大學
getting data from:   國立臺北科技大學
getting data from:   國立高雄科技大學
getting data from:   國立虎尾科技大學
getting data from:   國立澎湖科技大學
getting data from:   國立勤益科技大學
getting data from:   國立臺北護理健康大學
getting data from:   國立高雄餐旅大學
getting data from:   國立臺中科技大學
getting data from:   國立臺北商業大學
getting data from:   朝陽科技大學
getting data from:   南臺科技大學
getting data from:   崑山科技大學
getting data from:   嘉藥學校財團法人嘉南藥理大學
getting data from:   樹德科技大學
getting data from:   龍華科技大學
getting data from:   輔英科技大學
getting data from:   明新科技大學
getting data from:   弘光科技大學
getting data from:   健行科技大學
getting data from:   正修科技大學
getting data from:   萬能科技大學
getting data from:   建國科技大學
getting data from:   明志科技大學
getting data from:   高苑科技大學
getting data from:   大仁科技大學
getting data from:   聖約翰科技大學
getting data from:   嶺東科技大學
getting data from:   中國科技大學
getting data from:   中臺科技